In [1]:
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb

# Загружаем тестовые данные
test = pd.read_parquet('test.parquet')

In [2]:
# Функция для вычисления признаков (должна быть идентична той, что в обучении)
def compute_features(dates_array, values_array):
    values = np.array(values_array)
    features = {}
    features['mean'] = np.mean(values)
    features['std'] = np.std(values)
    features['max'] = np.max(values)
    features['min'] = np.min(values)
    features['median'] = np.median(values)
    features['skew'] = pd.Series(values).skew()
    features['kurtosis'] = pd.Series(values).kurtosis()
    features['length'] = len(values)
    return features

# Вычисляем признаки для тестовой выборки
test_features_list = []
for idx, row in test.iterrows():
    features = compute_features(row['dates'], row['values'])
    features['id'] = row['id']
    test_features_list.append(features)

test_features_df = pd.DataFrame(test_features_list)

In [3]:
print('\nТестовые признаки:')
print(test_features_df.head())


Тестовые признаки:
       mean       std   max   min  median      skew  kurtosis  length     id
0  0.037895  1.505985  3.13 -3.14   -0.01  0.067832 -0.342699      57   6125
1 -0.016909  1.249663  3.85 -2.24   -0.08  0.899283  1.058050      55  26781
2 -0.039569  1.000514  3.46 -1.45   -0.16  0.991187  1.296155      55  13333
3  0.159231  1.313112  3.43 -2.73    0.11  0.392145  0.022481      78  53218
4 -0.094705  1.243279  2.57 -2.66   -0.09 -0.033056 -0.479235      65  84204


In [4]:
# Загружаем сохраненную модель и масштабировщик
model = joblib.load('trained_model.pkl')
scaler = joblib.load('scaler.pkl')

In [5]:
# Подготовка данных для предсказания
X_test = test_features_df.drop(columns=['id'])
X_test_scaled = scaler.transform(X_test)

# Предсказание на тестовой выборке
test_pred_prob = model.predict(X_test_scaled, num_iteration=model.best_iteration)

# Формирование и сохранение submission файла
submission = pd.DataFrame({'id': test_features_df['id'], 'score': test_pred_prob})
submission.to_csv('submission.csv', index=False)

print('Submission файл сохранен как submission.csv')

Submission файл сохранен как submission.csv
